# 나이브 베이즈 분류(Naive Bayes Classifier)

##1 직접구현

### 1) Naive Bayes Classifier

In [22]:
training_set = [['me free lottery', 1],
 ['free get free you', 1],
 ['you free scholarship', 0],
 ['free to contact me', 0],
 ['you won award', 0],
 ['you ticket lottery', 1]]

### 2) 토큰 빈도수 및 문서별 토큰수 계산 (확률 계산을 위한 준비)

![대체 텍스트](https://wikimedia.org/api/rest_v1/media/math/render/svg/98f086c560aa2f66650060277dda4f90e54e30c0)

In [35]:
from collections import defaultdict

# 1: 스팸, 0:정상
doccnt0 = 0
doccnt1 = 0

wordfreq = defaultdict(lambda : [0, 0])
for doc, label in training_set :
  w_ls = doc.split()
  for w in w_ls :
    wordfreq[w][label] += 1

for k, (cnt0, cnt1) in wordfreq.items() :
  doccnt0 += cnt0
  doccnt1 += cnt1    

In [36]:
doccnt1



10

### 3) Training : 토큰별 조건부 확률 계산 

In [37]:
k = 0.5

wordprobs = defaultdict(lambda:[0,0])

for key, (cnt0, cnt1) in wordfreq.items() :
  wordprobs[key][0] = (cnt0+k)/(doccnt0 + 2*k)
  wordprobs[key][1] = (cnt1+k)/(doccnt1 + 2*k)

In [39]:
wordprobs

defaultdict(<function __main__.<lambda>>,
            {'award': [0.13636363636363635, 0.045454545454545456],
             'contact': [0.13636363636363635, 0.045454545454545456],
             'free': [0.22727272727272727, 0.3181818181818182],
             'get': [0.045454545454545456, 0.13636363636363635],
             'lottery': [0.045454545454545456, 0.22727272727272727],
             'me': [0.13636363636363635, 0.13636363636363635],
             'scholarship': [0.13636363636363635, 0.045454545454545456],
             'ticket': [0.045454545454545456, 0.13636363636363635],
             'to': [0.13636363636363635, 0.045454545454545456],
             'won': [0.13636363636363635, 0.045454545454545456],
             'you': [0.22727272727272727, 0.22727272727272727]})

### 4) Classify : 신규 텍스트가 주어졌을 때 확률 계산




In [40]:
import math

doc = "free lottery"
tokens = doc.split()

log_prob0 = log_prob1 = 0

for w, (p0, p1) in wordprobs.items() :
  if w in tokens :
    log_prob0 += math.log(p0)
    log_prob1 += math.log(p1)

log_prob0 += math.log(doccnt0/(doccnt0+doccnt1))
log_prob1 += math.log(doccnt1/(doccnt0+doccnt1))

prob0 = math.exp(log_prob0)
prob1 = math.exp(log_prob1)

print("정상 확률 : {}%".format(prob0/(prob0+prob1)))
print("스팸 확률 : {}%".format(prob1/(prob0+prob1)))

정상 확률 : 0.12500000000000008%
스팸 확률 : 0.8749999999999999%


In [33]:
log_prob0

-0.6931471805599453

In [34]:
log_prob1

-0.6931471805599453

0.00516528925619835

##2 sklearn 활용 (영문 뉴스 분류)

### 1) 뉴스 데이터 다운로드



In [41]:
from sklearn.datasets import fetch_20newsgroups
twenty_train = fetch_20newsgroups(subset='train', shuffle=True)
print(twenty_train.target_names) #뉴스 카테고리 출력
print(twenty_train.data[0]) #뉴데 이터 출력

['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']
From: lerxst@wam.umd.edu (where's my thing)
Subject: WHAT car is this!?
Nntp-Posting-Host: rac3.wam.umd.edu
Organization: University of Maryland, College Park
Lines: 15

 I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have o

In [42]:
twenty_train.data

["From: lerxst@wam.umd.edu (where's my thing)\nSubject: WHAT car is this!?\nNntp-Posting-Host: rac3.wam.umd.edu\nOrganization: University of Maryland, College Park\nLines: 15\n\n I was wondering if anyone out there could enlighten me on this car I saw\nthe other day. It was a 2-door sports car, looked to be from the late 60s/\nearly 70s. It was called a Bricklin. The doors were really small. In addition,\nthe front bumper was separate from the rest of the body. This is \nall I know. If anyone can tellme a model name, engine specs, years\nof production, where this car is made, history, or whatever info you\nhave on this funky looking car, please e-mail.\n\nThanks,\n- IL\n   ---- brought to you by your neighborhood Lerxst ----\n\n\n\n\n",
 "From: guykuo@carson.u.washington.edu (Guy Kuo)\nSubject: SI Clock Poll - Final Call\nSummary: Final call for SI clock reports\nKeywords: SI,acceleration,clock,upgrade\nArticle-I.D.: shelley.1qvfo9INNc3s\nOrganization: University of Washington\nLines: 

### 2) 문서 분류(파이프 라인 사용)

In [49]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB

text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                      ('clf', MultinomialNB())])

text_clf = text_clf.fit(twenty_train.data, twenty_train.target)


In [52]:
import numpy as np

test = fetch_20newsgroups(subset='test', shuffle=True)
predicted = text_clf.predict(test.data)
np.mean(predicted == test.target)

0.7738980350504514